In [1]:
import pyspark
sc = pyspark.SparkContext()

In [2]:
from sift.corpora import wikipedia
from sift.models import text, links

In [3]:
ls -l 20180301

total 1249080
-rw-rw-r--. 1 aborsu aborsu 159443563  3 mar 00:04 nlwiki-20180301-pages-articles1.xml-p1p224145.bz2
-rw-rw-r--. 1 aborsu aborsu 188462304  3 mar 00:09 nlwiki-20180301-pages-articles2.xml-p224146p776658.bz2
-rw-rw-r--. 1 aborsu aborsu 319879912  3 mar 00:18 nlwiki-20180301-pages-articles3.xml-p776661p1874903.bz2
-rw-rw-r--. 1 aborsu aborsu 293193701  3 mar 00:32 nlwiki-20180301-pages-articles4.xml-p1874904p3374904.bz2
-rw-rw-r--. 1 aborsu aborsu 296945495  3 mar 00:59 nlwiki-20180301-pages-articles4.xml-p3374904p4874904.bz2
-rw-rw-r--. 1 aborsu aborsu  21096327  3 mar 00:35 nlwiki-20180301-pages-articles4.xml-p4874904p4955578.bz2


In [4]:
import os
base_path = os.path.join(os.getcwd(), '20180301')

In [5]:
corpus = wikipedia.WikipediaCorpus().build(sc, base_path)

In [6]:
a = corpus.take(10)[3]

In [7]:
from sift.corpora import wikicorpus
pfx = wikicorpus.wikilink_prefix
redirects = corpus.filter(lambda page: page['redirect'] != None)



{'_id': 'en.wikipedia.org/wiki/Kobaltbom',
 'content': None,
 'namespace': '0',
 'pid': 4874904,
 'redirect': 'Kobalt'}

In [8]:
redirects = wikipedia.WikipediaRedirects()(corpus)
docs = wikipedia.WikipediaArticles()(corpus, redirects)

In [9]:
redirects.first()

{'_id': 'en.wikipedia.org/wiki/Lijst_van_gemeentelijke_monumenten_in_Leiden,_straat_G-I',
 'target': 'en.wikipedia.org/wiki/Lijst_van_gemeentelijke_monumenten_in_Leiden'}

In [ ]:
docs.first()

In [ ]:
vocab = text.TermVocab(max_rank=100000,min_rank=0,lowercase=True,min_df=5)(docs.sample(False, 0.25))

In [ ]:
vocab.take(10)

In [ ]:
text.EntityMentions(sentence_window=1,lowercase=True)(docs).take(1)

In [ ]:
mentions = text.IndexMappedMentions(sentence_window=1,lowercase=True)(sc, docs, vocab)

In [ ]:
mentions.take(1)

In [ ]:
links.EntityCounts()(docs).take(10)

In [ ]:
links\
    .EntityNameCounts()(docs)\
    .filter(lambda r: len(r['counts']) >= 3)\
    .take(5)